In [7]:
LOSS_TYPE = 'BCE'
IS_KUNI = False
KUNI_AGG = 'mean'
KUNI_LAM = 1
IS_SEP_LOSS = False
CFG = 'cfgs/brain/brain_pospoolxyz.yaml'#'cfgs/brain/brain_pointwisemlp.yaml'
DEVICE = 1
NUM_VOTES = 3
BRAINS = range(26)
REPEAT = 5

In [8]:
from utils.fcd import *
from utils.crop import BrainDataSegCrop
torch.cuda.set_device(DEVICE)
torch.cuda.current_device()

1

In [9]:
config = config_seting(CFG)

In [10]:
def model_and_brains_to_points_with_predictions(config,model = None, brains = [0], num_votes = 10, repeat = 1):
    res = {}
    for brain_num in tqdm(brains):
        data_loader = BrainDataSegCrop(num_points = 4096,task = 'test',test_brain = brain_num,is_return_center = True)
        model, criterion = build_multi_part_segmentation(config = config,
                                                         weights = None,
                                                         type = LOSS_TYPE,
                                                         is_kuni = IS_KUNI,
                                                         kuni_agg = KUNI_AGG,
                                                         kuni_lam = KUNI_LAM,
                                                         is_sep_loss = IS_SEP_LOSS
                                                        )
        model.cuda()
        model.load_state_dict(torch.load(f'checkpoints/2_{brain_num}.pth'))
        criterion.cuda()
        model.eval()

        with torch.no_grad():
            pred_soft_flats = []
            points_labels_flats = []
            center_coords_flats = []
            points_orig_flats = []
            all_logits = []
            all_points_labels = []
            all_shape_labels = []
            all_masks = []
            TS = d_utils.BatchPointcloudScaleAndJitter(scale_low=config.scale_low,
                                                       scale_high=config.scale_high,
                                                       std=config.noise_std,
                                                       clip=config.noise_clip)
            m = torch.nn.Softmax(dim=0)
            for _ in range(repeat):
                for crop in data_loader:
                    points_orig, mask, points_labels, shape_labels, center_coords = crop
                    vote_logits = None
                    vote_points_labels = None
                    vote_shape_labels = None
                    vote_masks = None
                    points_orig = points_orig.unsqueeze(0)
                    mask = mask.unsqueeze(0)
                    points_labels = points_labels.unsqueeze(0)
                    shape_labels = shape_labels.unsqueeze(0)
                    preds = []
                    for v in range(num_votes):
                        batch_logits = []
                        batch_points_labels = []
                        batch_shape_labels = []
                        batch_masks = []

                        if v > 0:
                            points = TS(points_orig)
                        else:
                            points = points_orig
                        # forward
                        features = points
                        features = features.transpose(1, 2).contiguous()
                        points = points[:,:,:3].cuda(non_blocking=True)
                        mask = mask.cuda(non_blocking=True)
                        features = features.cuda(non_blocking=True)
                        points_labels = points_labels.cuda(non_blocking=True)
                        shape_labels = shape_labels.cuda(non_blocking=True)
                        pred = model(points, mask, features)
                        preds.append(pred[0])
                    preds = torch.cat(preds).mean(dim = 0)
                    pred_soft_flats += list(np.array(m(preds)[1,:].reshape(-1).detach().cpu()))
                    points_labels_flats += list(np.array(points_labels.reshape(-1).detach().cpu()))
                    points_orig = points_orig.squeeze(0)
                    points_orig_flats += list(((points_orig[:,:3])*32+32).detach().cpu().numpy()+np.array(center_coords))
        tmp = (np.array(points_orig_flats),np.array(pred_soft_flats),points_labels_flats)
#         confs = confusion_matrix(tmp[2], tmp[1]>0.5)
#         msIoU = confs[1,1]/(confs[1,1]+confs[1,0]+confs[0,1])
#         dice_score = 2 * confs[1,1] / (2 * confs[1,1]+confs[1,0]+confs[0,1])
#         print(f'Confision matrix: \n{confs}')
#         print(f'IoU: {msIoU}')
#         print(f'DICE Score: {dice_score}')
        res[brain_num] = tmp
    return res

In [11]:
res_dict = model_and_brains_to_points_with_predictions(config = config, num_votes = NUM_VOTES, brains = BRAINS,repeat = REPEAT)
np.save('for_visualisation.npy',res_dict)


  0%|          | 0/26 [00:00<?, ?it/s]
Exception in thread Thread-3:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_monitor.py", line 62, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 26/26 [2:58:04<00:00, 410.95s/it]  


In [18]:
def result_to_metrics(res):
    conf = confusion_matrix(res[2], res[1]>0.5)
    IoU = conf[1,1]/(conf[1,1]+conf[1,0]+conf[0,1])
    dice = 2 * conf[1,1] / (2 * conf[1,1]+conf[1,0]+conf[0,1])
    d_in = np.dot(res[2],res[1])/np.sum(res[2])
    d_out = np.dot(np.ones(len(res[2]))-np.array(res[2]),res[1]) / (len(res[2]) - np.sum(res[2]))
    contrast = (d_in - d_out) /  (d_in + d_out)
    roc = roc_auc_score(res[2],res[1])
    return conf,IoU,dice,contrast, roc

In [19]:
res_metrics = {}
for brain in tqdm(res_dict):
    res_metrics[brain] = {}
    res = res_dict[brain]
    res_metrics[brain]['conf_all'],res_metrics[brain]['IoU_all'],res_metrics[brain]['dice_all'],res_metrics[brain]['contrast_all'],res_metrics[brain]['roc_all'] = result_to_metrics(res)
    len_1_part = len(res[0]) // REPEAT
    res_metrics[brain]['confs'] = []
    res_metrics[brain]['IoUs'] = []
    res_metrics[brain]['dices'] = []
    res_metrics[brain]['contrasts'] = []
    res_metrics[brain]['roc'] = []
    for repeat in range(REPEAT):
        res_part = [res[i][len_1_part*repeat:len_1_part*(repeat+1)] for i in range(3)]
        conf,IoU,dice, contrast,roc = result_to_metrics(res_part)
        res_metrics[brain]['confs'].append(conf)
        res_metrics[brain]['IoUs'].append(IoU)
        res_metrics[brain]['dices'].append(dice)
        res_metrics[brain]['contrasts'].append(contrast)
        res_metrics[brain]['roc'].append(roc)
        

100%|██████████| 26/26 [06:14<00:00, 14.40s/it]


In [21]:
for brain in res_metrics:
    print(brain,res_metrics[brain]['dice_all'])

0 0.6139953965100038
1 0.014809041309431021
2 0.0
3 0.7093718235413702
4 0.0
5 0.07088668760570185
6 0.3825973932194823
7 0.0
8 0.0
9 0.10185317517031109
10 0.0
11 0.25831820931639443
12 0.21067221067221067
13 0.004460019114367633
14 0.005105762217359592
15 0.0
16 0.0
17 0.0
18 0.0
19 0.11840719151625817
20 0.0
21 0.0
22 0.0
23 0.0
24 0.0
25 0.0
